# Concurrency with asyncio
For real parallelism, you must have multiple cores. A modern laptop has four CPU cores but is routinely running more than 100 processes at any given time under normal, casual use. So, in practice, most processing happens concurrently and not in parallel.

This chapter introduces asyncio, a package that implements concurrency with coroutines driven by an event loop. It’s one of the largest and most ambitious libraries ever added to Python. Guido van Rossum developed asyncio outside of the Python repos‐
itory and gave the project a code name of 'Tulip' — so you’ll see references to that flower when researching this topic online.

## Thread Versus Coroutine: A Comparison


In [1]:
import threading
import itertools
import time
import sys

class Signal:
    go = True

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))

def slow_function():
    # pretend waiting a long time for I/O
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main():
    result = supervisor()
    print('Answer:', result)

if __name__ == '__main__':
    main()


spinner object: <Thread(Thread-6, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking          Answer: 42


I also like the use of itertools cycle.

In [27]:
class A:
    def __init__(self):
        self.my_list = list(range(3))
        self.finished = False

    def __call__(self):
        if len(self.my_list) > 0:
            return self.my_list.pop()
        else:
            self.finished = True
            return None


In [28]:
import itertools

a = A()
b = A()
for obj in itertools.cycle([a, b]):
    value = obj()
    print(value)
    if a.finished and b.finished:
        break


2
2
1
1
0
0
None
None


Now we take a similar approach with a coroutine...

In [2]:
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))


@asyncio.coroutine
def slow_function():
    # pretend waiting a long time for I/O
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)

if __name__ == '__main__':
    main()


RuntimeError: This event loop is already running

spinner object: <Task pending coro=<spin() running at <ipython-input-2-0a7960cb7659>:5>>
| thinking/ thinking!

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: asyncio.async() function is deprecated, use ensure_future()


- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking          

See the [docs](https://docs.python.org/3/library/asyncio.html) for details.

The event loop time is precious. If you are not making progress, you should step off the loop, so that someone else can. Event loop is the measure of progress. Coroutines (co-operative routines) are a key element of the symphony. It is the coroutines, and their co-operative nature, that enables giving up control of the event loop, when the coroutine has nothing useful to do.

In Python 3.5+, the way a coroutine pauses itself is using the await keyword. Inside a coroutine, when you await on another coroutine, you step off the event loop and schedule the awaited coroutine to run immediately. That is, an await other_coroutine inside a coroutine will pause it, and schedule the coroutine other_coroutine to run immediately.

In [3]:
import asyncio

# definition of a coroutine
async def coroutine_1():
    print('coroutine_1 is active on the event loop')

    print('coroutine_1 yielding control. Going to be blocked for 4 seconds')
    await asyncio.sleep(4)

    print('coroutine_1 resumed. coroutine_1 exiting')


# definition of a coroutine
async def coroutine_2():
    print('coroutine_2 is active on the event loop')

    print('coroutine_2 yielding control. Going to be blocked for 5 seconds')
    await asyncio.sleep(5)

    print('coroutine_2 resumed. coroutine_2 exiting')


# this is the event loop
loop = asyncio.get_event_loop()

# schedule both the coroutines to run on the event loop
loop.run_until_complete(asyncio.gather(coroutine_1(), coroutine_2()))

RuntimeError: This event loop is already running

coroutine_1 is active on the event loop
coroutine_1 yielding control. Going to be blocked for 4 seconds
coroutine_2 is active on the event loop
coroutine_2 yielding control. Going to be blocked for 5 seconds
coroutine_1 resumed. coroutine_1 exiting
coroutine_2 resumed. coroutine_2 exiting


There is a thread on github about the [RuntimeError](https://github.com/spyder-ide/spyder/issues/7096). There’s no API to terminate a thread from the outside, because a thread could be interrupted at any point, leaving the system in an invalid state. For tasks, there is the Task.cancel() instance method, which raises CancelledError inside the coroutine. 

***